# Data management with datatable in python

In this example, we will be using the python version of datatable which is available [here](https://github.com/h2oai/datatable). This library can be directly installed using pip. Notice that you need to have python 3.5>= and a relative modern version of pip, otherwise you may see an error like what happened [here](https://github.com/h2oai/datatable/issues/2268).

In [1]:
import datatable as dt;
import numpy as np;
import matplotlib.pyplot as mp; # To call the plot function on matplotlib
import pandas as pd;
import ftplib;

In [2]:
stations = dt.fread("ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv")

In [3]:
stations.head(5)

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,007018,99999,WXPOD 7018,,,,0,0,7018,20110309,20130730
1,007026,99999,WXPOD 7026,AF,,,0,0,7026,20120713,20170822
2,007070,99999,WXPOD 7070,AF,,,0,0,7070,20140923,20150926
3,008260,99999,WXPOD8270,,,,0,0,0,20050101,20100920
4,008268,99999,WXPOD8278,AF,,,32.95,65.567,1156.7,20100519,20120323


We now need to filter the data and restrict to continental US only

In [10]:
to_exclude = ["AK", "HI", "", "PR", "VI"]
st_us = stations[
    (dt.f.CTRY == 'US') & 
    ~np.isin(stations[:,dt.f.STATE], to_exclude) &
    (dt.f.WBAN < 99999) & (dt.f.USAF != '999999') &
    ~dt.f['STATION NAME'].re_match('BUOY|ISLAND|PLATFORM'),
    :]
# Checking the size
st_us.shape

(2234, 11)

In [11]:
# Total number of records per state
st_us[:, {'total' : dt.count()}, dt.by('STATE')]


,STATE,total
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,AL,40
1,AR,37
2,AZ,46
3,CA,161
4,CO,62
5,CT,10
6,DE,4
7,FL,93
8,GA,63


In [12]:
# Further operations
st_us[:, 'BEGIN_YR'] = np.floor(st_us[:, dt.f.BEGIN/10000.0])
st_us[:, 'END_YR']   = np.floor(st_us[:, dt.f.END/10000.0])
st_us[:, 'USAF']     = np.int64(
    np.where(
        st_us[:,dt.f.USAF.re_match('^[a-zA-Z].+')],
        -9999,
        st_us[:,dt.f.USAF]
    )
)
st_us = st_us[dt.f.USAF >= 0, :]

In [13]:
st_us[(dt.f.END_YR >= 2009) & (dt.f.BEGIN_YR <= 2009),:]

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,BEGIN_YR,END_YR
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,690150,93121,TWENTY NINE PALMS,US,CA,KNXP,34.3,−116.167,625.1,19900102,20200731,1990,2020
1,690190,13910,ABILENE DYESS AFB,US,TX,KDYS,32.433,−99.85,545.3,19431201,20091231,1943,2009
2,690230,24255,WHIDBEY ISLAND NAS,US,WA,KNUW,48.35,−122.667,14.3,19891201,20090602,1989,2009
3,699604,3145,YUMA MCAS,US,AZ,KNYL,32.65,−114.617,64.9,19870701,20091231,1987,2009
4,720110,53983,LLANO MUNICIPAL AIRPORT,US,TX,KAQO,30.784,−98.662,335.9,20050101,20200731,2005,2020
5,720113,54829,OAKLAND/TROY AIRPORT,US,MI,KVLL,42.543,−83.178,218.2,20050101,20200731,2005,2020
6,720120,63837,HILTON HEAD AIRPORT,US,SC,KHXD,32.217,−80.7,7.3,20060101,20200731,2006,2020
7,720137,4867,MORS MUNI-J.R. WSBRN FD AP,US,IL,KC09,41.425,−88.419,178,20060101,20200731,2006,2020
8,720141,4868,MARSHALL CO,US,IL,KC75,41.019,−89.386,173.1,20060101,20130430,2006,2013


In [ ]:
import tempfile;
tmp = tempfile.NamedTemporaryFile(suffix=".gz");
ftp = ftplib.FTP("ftp.ncdc.noaa.gov") 

In [ ]:
ftp.login();
ftp.cwd("/pub/data/noaa/2019/")

In [ ]:
with open(tmp.name, "wb") as f:
    ftp.retrbinary('RETR 720538-00164-2019.gz', f.write)


In [ ]:
cwidths = [4, 6, 5, 4, 2, 2, 2, 2, 1, 6, 7, 5, 5, 5, 4, 3, 1, 1, 4, 1, 5, 1, 1, 1, 6, 1, 1, 1, 5, 1, 5, 1, 5, 1]
cnames  = ["ID","USAFID", "WBAN", "year", "month","day", "hour", "min","srcflag", "lat",
  "lon", "typecode","elev","callid","qcname","wind.dir", "wind.dir.qc", 
  "wind.type.code","wind.sp","wind.sp.qc", "ceiling.ht","ceiling.ht.qc",
  "ceiling.ht.method","sky.cond","vis.dist","vis.dist.qc","vis.var","vis.var.qc",
  "temp","temp.qc", "dew.point","dew.point.qc","atm.press","atm.press.qc"]
dat = pd.read_fwf( 
    tmp.name, #"ftp://ftp.ncdc.noaa.gov/pub/data/noaa/2019/720538-00164-2019.gz",
    widths = cwidths,
    names  = cnames
);

In [ ]:
dat

In [ ]:
for i in [['a', 'b'],'b','c']:
    print ("Hello " + str(i));

In [ ]:
met = dt.fread("met_all_dt.gz")

In [ ]:
met

Average temperature by hour of the day for the first day of the month. The `dt.f` method allows accessing column names directly.

In [ ]:
averages = met[dt.f.day == 1,:][:, dt.mean(dt.f.temp),dt.by("hour")]

In [ ]:
# This is how you can get the names of the month
met.names

In [ ]:
averages = averages.to_numpy()

In [ ]:
mp.plot(averages[:,0], averages[:,1])